In [1]:
import pandas as pd
import numpy as np
from PIL import Image
from PIL import ImageOps
import PIL
import torch, torchvision
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import math
import random

In [2]:
columns = ['image_index', 'confidence', 'x1','y1','x2','y2','class']
img_path='/home/rliu/yolo2/v2_pytorch_yolo2/data/an_data/VOCdevkit/VOC2007/JPEGImages/'

In [103]:
index_train = np.arange(0)
df_train = pd.DataFrame(index=index_train)

pos = pd.read_csv('/home/rliu/github/defect_classifier/yolo2_dm/results/train/comp4_det_test_pos.txt',header=None, sep=' ')
pos['class'] = 0
neg = pd.read_csv('/home/rliu/github/defect_classifier/yolo2_dm/results/train/comp4_det_test_neg.txt',header=None, sep=' ')
neg['class'] = 1
pos_o = pd.read_csv('/home/rliu/github/defect_classifier/yolo2_dm/results/train/comp4_det_test_pos_o.txt',header=None, sep=' ')
pos_o['class'] = 2
nuc = pd.read_csv('/home/rliu/github/defect_classifier/yolo2_dm/results/train/comp4_det_test_nuc.txt',header=None, sep=' ')
nuc['class'] = 3
df_train = df_train.append(pos)
df_train = df_train.append(neg)
df_train = df_train.append(pos_o)
df_train = df_train.append(nuc)
df_train.columns = columns
df_train = df_train.sort_values('image_index')
df_train = df_train.reset_index()
df_train = df_train.drop(columns='index')
for index, row in df_train.iterrows():
    img = Image.open(img_path + '%06.0f.jpg' % row.image_index)
    width, height = img.size
    row.x1 = row.x1/width
    row.x2 = row.x2/width
    row.y1 = row.y1/height
    row.y2 = row.y2/height
    df_train.at[index] = row
for index, row in df_train.iterrows(): # filter boxes that are too big or 
    if (row.x2-row.x1)*(row.y2-row.y1) > 0.2:
        df_train = df_train.drop(index)

In [106]:
df_train = df_train.reset_index()
df_train = df_train.drop(columns='index')

In [107]:
df_train.to_csv('/home/rliu/github/defect_classifier/yolo2_dm/results/train_yolo.csv', sep=' ', index=False)

In [104]:
index_test = np.arange(0)
df_test = pd.DataFrame(index=index_test)

pos = pd.read_csv('/home/rliu/github/defect_classifier/yolo2_dm/results/test/comp4_det_test_pos.txt',header=None, sep=' ')
pos['class'] = 0
neg = pd.read_csv('/home/rliu/github/defect_classifier/yolo2_dm/results/test/comp4_det_test_neg.txt',header=None, sep=' ')
neg['class'] = 1
pos_o = pd.read_csv('/home/rliu/github/defect_classifier/yolo2_dm/results/test/comp4_det_test_pos_o.txt',header=None, sep=' ')
pos_o['class'] = 2
nuc = pd.read_csv('/home/rliu/github/defect_classifier/yolo2_dm/results/test/comp4_det_test_nuc.txt',header=None, sep=' ')
nuc['class'] = 3
df_test = df_test.append(pos)
df_test = df_test.append(neg)
df_test = df_test.append(pos_o)
df_test = df_test.append(nuc)
df_test.columns = columns
df_test = df_test.sort_values('image_index')
df_test = df_test.reset_index()
df_test = df_test.drop(columns='index')
for index, row in df_test.iterrows():
    img = Image.open(img_path + '%06.0f.jpg' % row.image_index)
    width, height = img.size
    row.x1 = row.x1/width
    row.x2 = row.x2/width
    row.y1 = row.y1/height
    row.y2 = row.y2/height
    df_test.at[index] = row
for index, row in df_test.iterrows():
    if (row.x2-row.x1)*(row.y2-row.y1) > 0.2:
        df_test = df_test.drop(index)

In [108]:
df_test = df_test.reset_index()
df_test = df_test.drop(columns='index')

In [109]:
df_test.to_csv('/home/rliu/github/defect_classifier/yolo2_dm/results/test_yolo.csv', sep=' ', index=False)

In [10]:
df_labels = pd.read_csv('/home/rliu/yolo2/v2_pytorch_yolo2/data/an_data/VOCdevkit/VOC2007/csv_labels/train.csv', sep=" ")
df_yolo = pd.read_csv('/home/rliu/github/defect_classifier/yolo2_dm/results/train_yolo.csv', sep=' ')
n_samples = 1995
non_pos_ratio = 10
non_inner_circle = 0.02
non_outer_circle = 0.07
method = 'yolo'

df_pos = df_labels[df_labels['class'] == 0]
df_neg = df_labels[df_labels['class'] == 1]
df_pos_o = df_labels[df_labels['class'] == 2]
df_nuc = df_labels[df_labels['class'] == 3]
frames = [df_pos.sample(n=n_samples), df_neg.sample(n=n_samples), df_pos_o.sample(n=n_samples), df_nuc.sample(n=n_samples)]
df_labels_samples = pd.concat(frames)
# print(df_labels_non.head(10))
if method=='hard':
    df_labels_non = df_labels_samples.sample(n=n_samples*non_pos_ratio)
    for index, row in df_labels_non.iterrows():
        min_dis = 0  # make sure go in for loop
        while min_dis < non_inner_circle:
            dx,dy = sample_point_circular(non_inner_circle, non_outer_circle)
            new_point = [row.x+dx, row.y+dy]
            df_image = df_labels[df_labels['image_index'] == row['image_index']] # retrive all points in the image
            min_dis = 1
            for index_im, row_im in df_image.iterrows(): # check distance from new_point to each point, if smaller than threshold than thow away
                dis = math.sqrt(math.pow(new_point[0]-row.x,2)+math.pow(new_point[1]-row.y,2))
                if dis < min_dis:
                    min_dis = dis
        df_labels_non.at[index,'x'] = new_point[0]
        df_labels_non.at[index,'y'] = new_point[1]
        df_labels_non.at[index,'class'] = 4
elif method=='uniform':
    df_labels_non = df_labels_samples.sample(n=n_samples*non_pos_ratio)
    for index, row in df_labels_non.iterrows():
        min_dis = 0
        while min_dis < non_inner_circle:
            new_point = [random.random(),random.random()]
            df_image = df_labels[df_labels['image_index'] == row['image_index']]
            min_dis = 1
            for index_im, row_im in df_image.iterrows():
                dis = math.sqrt(math.pow(new_point[0]-row.x,2)+math.pow(new_point[1]-row.y,2))
                if dis < min_dis:
                    min_dis = dis
        df_labels_non.at[index,'x'] = new_point[0]
        df_labels_non.at[index,'y'] = new_point[1]
        df_labels_non.at[index,'class'] = 4
elif method=='yolo':
    columns = ['image_index','class', 'x', 'y']
    df_labels_non = pd.DataFrame(index=np.arange(2*n_samples*non_pos_ratio), columns=columns)
    df_labels_non['class'] = 4 # background class
    df_yolo_samples = df_yolo.sample(n=2*n_samples*non_pos_ratio)
    df_yolo_samples = df_yolo_samples.reset_index() #A reset index
    df_yolo_samples = df_yolo_samples.drop(columns='index')
    for index, row in df_yolo_samples.iterrows():
        pts = sample_rec(row) # sample one point from a bounding box
        df_labels_non.at[index,'x'] = pts[0]
        df_labels_non.at[index,'y'] = pts[1]
        df_labels_non.at[index,'image_index'] = row['image_index']
    print(df_labels_non.shape[0])
    df_labels_non = df_labels_non.reset_index()
    df_labels_non = df_labels_non.drop(columns='index')
    for index_non, row_non in df_labels_non.iterrows(): # check if points are too close to a defect
        df_image = df_labels[df_labels['image_index'] == row_non['image_index']] # extract all defect labels
        min_dis = 1
        for index_im, row_im in df_image.iterrows():
            dis = math.sqrt(math.pow(row_non.x-row_im.x,2)+math.pow(row_non.x-row_im.y,2))
            if dis < min_dis:
                min_dis = dis
#         print(min_dis)
        if min_dis < non_inner_circle:
            df_labels_non = df_labels_non.drop(index_non)
    df_labels_non = df_labels_non.sample(n=n_samples*non_pos_ratio)
df_labels_samples = df_labels_samples.append(df_labels_non)
df_labels_samples = df_labels_samples.reset_index()
df_labels_samples = df_labels_samples.drop(['index'],axis = 1)
print(df_labels_non.shape[0])

39900
19950


In [8]:
df_labels_samples

,image_index,class,x,y
0,3421,0,0.355978,0.581551
1,8635,0,0.88125,0.35
2,3855,0,0.732337,0.264706
3,3972,0,0.532609,0.399733
4,198,0,0.375,0.794118
5,3337,0,0.427989,0.51738
6,1083,0,0.505435,0.236631
7,1190,0,0.519022,0.370321
8,413,0,0.252717,0.566845
9,8792,0,0.789583,0.672917


In [244]:
df_yolo = pd.read_csv('/home/rliu/github/defect_classifier/yolo2_dm/results/test_yolo.csv', sep=' ')
yolo_pos = df_yolo[df_yolo['class'] == 0]
yolo_neg = df_yolo[df_yolo['class'] == 1]
yolo_pos_o = df_yolo[df_yolo['class'] == 2]
yolo_nuc = df_yolo[df_yolo['class'] == 3]

In [253]:
print(df_pos.shape[0], df_neg.shape[0], df_pos_o.shape[0], df_nuc.shape[0])

16102 7614 4719 1995


In [238]:
df_yolo[df_yolo['image_index']==2444]

,image_index,confidence,x1,y1,x2,y2,class
24277,2444.0,0.003805,0.010018,0.213950,0.232555,0.477831,3.0
24278,2444.0,0.003142,0.022032,0.003618,0.204615,0.065576,3.0
24279,2444.0,0.003444,0.004814,0.444268,0.207200,0.696273,3.0
24280,2444.0,0.894145,0.343890,0.617170,0.560239,0.838082,2.0
24281,2444.0,0.007977,0.491333,-0.001837,0.667777,0.109138,1.0
24282,2444.0,0.827867,0.672746,0.078359,0.835590,0.246004,1.0
24283,2444.0,0.857882,0.441225,0.798254,0.609274,0.968963,1.0
24284,2444.0,0.004814,0.802269,0.729822,0.953744,0.877076,1.0
24285,2444.0,0.785586,0.539523,0.640136,0.701722,0.796519,0.0
24286,2444.0,0.832491,0.738215,0.324597,0.900149,0.485607,0.0


In [208]:
df_labels_non = df_labels_non.reset_index()
df_labels_non = df_labels_non.drop(columns='index')
for index_non, row_non in df_labels_non.iterrows(): # check if points are too close to a defect
    df_image = df_labels[df_labels['image_index'] == row_non['image_index']] # extract all defect labels
    min_dis = 1
#         print(df_image)
    for index_im, row_im in df_image.iterrows():
        dis = math.sqrt(math.pow(row_non.x-row_im.x,2)+math.pow(row_non.x-row_im.y,2))
        if dis < min_dis:
            min_dis = dis
#         print(min_dis)
    if min_dis < non_inner_circle:
        df_labels_non = df_labels_non.drop(index_non)

In [252]:
df_labels_non.sort_values(by='image_index', ascending=True)

,image_index,class,x,y
16339,1,4,0.252029,0.352851
16035,1,4,0.313406,0.415301
17134,2,4,0.53309,0.663172
19346,2,4,0.494618,0.903456
14691,2,4,0.267687,0.736261
8940,3,4,0.201864,0.700881
10924,3,4,0.1395,0.631402
17632,3,4,0.401677,0.871561
14567,4,4,0.501657,0.872133
15844,4,4,0.0951958,0.550333


In [207]:
print(df_labels.shape[0])

30428


In [163]:
# for a point from bounding boxes in the same image
def findPts(df = df_yolo, df_labels = df_labels, point = x, num = 10, non_inner_circle = 0.02):
    df = df.loc[df['image_index'] == x['image_index']] # extract boxes from the same image
    index_total = np.arange(0)
    columns = ['image_index','class', 'x', 'y']
    df_non = pd.DataFrame(index=index_total, columns=columns)
    for index, row in df.iterrows(): # for each box
        df_pts = sample_rec(row=row) # sample some points from a box depending on its area
        df_non = df_non.append(df_pts)
    for index_non, row_non in df_non.iterrows(): # check if points are too close to a defect
        df_image = df_labels[df_labels['image_index'] == row_non['image_index']] # extract all defect labels
        min_dis = 1
        for index_im, row_im in df_image.iterrows():
            dis = math.sqrt(math.pow(row_non.x-row_im.x,2)+math.pow(row_non.x-row_im.y,2))
            if dis < min_dis:
                min_dis = dis
        if min_dis < non_inner_circle:
            df_non.drop(index_non)
    df_non = df_non.sample(n=num)
    return df_non
    

In [9]:
def sample_rec(row):
    width = row.x2 - row.x1
    height = row.y2 - row.y1
    pts = [random.random(),random.random()]
    pts[0] = pts[0] * width + row.x1 # sample points from bounding boxes
    pts[1] = pts[1] * height + row.y1
    return pts

In [183]:
def sample_rec(row = row):
    width = row.x2 - row.x1
    height = row.y2 - row.y1
    n_samples=math.floor(height*width*100) # decide how many points to sample from one box
    pts=np.random.rand(n_samples,2)
    pts[:,0] = pts[:,0] * width + row.x1 # sample points from bounding boxes
    pts[:,1] = pts[:,1] * height + row.y1
    columns = ['image_index','class', 'x', 'y']
    df_pts = pd.DataFrame(index=np.arange(n_samples), columns=columns)
    df_pts['image_index'] = row.image_index
    df_pts['class'] = 4
    df_pts.x = pts[:,0]
    df_pts.y = pts[:,1]
    return df_pts

In [115]:
def create_circular_mask(h, w, center=None, radius=None):

    if center is None: # use the middle of the image
        center = [int(w/2), int(h/2)]
    if radius is None: # use the smallest distance between the center and image walls
        radius = min(center[0], center[1], w-center[0], h-center[1])

    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

    mask = dist_from_center <= radius
    mask = mask.astype(int)
    return mask

In [117]:
def sample_point_circular(circle_min = 0.02, circle_max = 0.07):
    # random angle
    alpha = 2 * math.pi * random.random()
    # random radius
    r = (circle_max - circle_min) * random.random() + circle_min
    # calculating coordinates
    x = r * math.cos(alpha)
    y = r * math.sin(alpha)
    return x,y

In [217]:
width = row.x2 - row.x1
height = row.y2 - row.y1
pts=np.random.rand(2,1)
pts[0] = pts[0] * width + row.x1 # sample points from bounding boxes
pts[1] = pts[1] * height + row.y1


In [218]:
pts

array([[0.62444626],
       [0.0611637 ]])

In [159]:
findPts(df = df_yolo, df_labels = df_labels, point = x, num = 10, non_inner_circle = 0.02)

,image_index,class,x,y
6,1231.0,4,0.418376,0.057364
3,1231.0,4,0.437097,0.159426
8,1231.0,4,0.370435,0.345227
5,1231.0,4,0.838667,0.242256
3,1231.0,4,0.857338,0.261789
9,1231.0,4,0.537624,0.136490
4,1231.0,4,0.638137,0.070268
5,1231.0,4,0.470187,0.080757
8,1231.0,4,0.612439,0.164186
0,1231.0,4,0.669879,0.634487


In [213]:
math.sqrt(0.01)*748

74.8

In [57]:
row

image_index    1493.000000
confidence        0.002587
x1                0.644387
y1                0.045820
x2                0.800387
y2                0.200559
class             1.000000
Name: 14989, dtype: float64

In [56]:
pts

array([[0.66348719, 0.12115096],
       [0.7150121 , 0.13109465],
       [0.6631527 , 0.06530974],
       [0.66543766, 0.08111272],
       [0.66261188, 0.15030722],
       [0.72617087, 0.10787595],
       [0.79038186, 0.15458732],
       [0.75613897, 0.10187293],
       [0.7898342 , 0.12257147],
       [0.72598564, 0.1155962 ],
       [0.67146938, 0.17577441],
       [0.71051811, 0.08834275]])

In [45]:
df = df_train.loc[df_train['image_index'] == x.image_index]

In [47]:
for index, row in df.iterrows():
    print row

image_index    3585.000000
confidence        0.004106
x1               49.758846
y1              107.163834
x2              219.627106
y2              279.530426
class             3.000000
Name: 35458, dtype: float64
image_index    3585.000000
confidence        0.007326
x1              189.714905
y1              610.155762
x2              315.364471
y2              723.965088
class             1.000000
Name: 35459, dtype: float64
image_index    3585.000000
confidence        0.016895
x1              637.707336
y1              316.623322
x2              736.824219
y2              448.321014
class             1.000000
Name: 35460, dtype: float64
image_index    3585.000000
confidence        0.002958
x1              682.960266
y1                2.644402
x2              733.811646
y2               54.404205
class             1.000000
Name: 35461, dtype: float64
image_index    3585.000000
confidence        0.004826
x1              306.767242
y1              654.178528
x2              436.5600

In [56]:
height = row.x2 - row.x1
width = row.y2 - row.y1
n_samples = math.floor((height * width)/1000)


In [58]:
height, width

(125.33706700000002, 120.02255300000002)